# Imports

In [1]:
import bruce, numpy as np, matplotlib.pyplot as plt, os
from astropy.table import Table, Column
from astropy.stats import sigma_clip
from astropy.time import Time , TimeDelta
from astropy import units as u
import gc, multiprocess
from scipy.stats import median_abs_deviation

/Users/sam/anaconda3/lib/python3.9/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/sam/.lightkurve/cache. Please move all the files in the legacy directory /Users/sam/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(


In [2]:
gc.collect()

0

# Load the tics

Data should have the form of (ascii or CSV, your choice). t_zero is in full BJD, width is in days, depth is in normalised flux. If you ue SPOCFIT, if you fit a single transit you will see 3 reported values on the bottom row which are these values for each event (t_zero, width, depth).

tic_id,	t_zero_1,	width_1,	depth_1,	t_zero_2,	width_2,	depth_2,

In [3]:
duos = Table.read('duos.csv')
duos['tic_id'] = duos['TIC ID']
print(duos)

  TIC ID  Linked TIC ID  TOI ID   t_zero_1  ... SG1 Published?   tic_id 
--------- ------------- ------- ----------- ... --- ---------- ---------
 20904104      20904104       - 2458390.178 ...  --         --  20904104
 22317640      22317640       - 2458589.378 ...  --         --  22317640
 42428568      42428568       -  2458475.69 ...  --         --  42428568
 67599025      67599025       - 2458404.028 ...  --         --  67599025
 71028120      71028120       - 2458445.866 ...  --         --  71028120
 71272316      71272316       - 2459070.725 ...  --         --  71272316
107113345     107113345       - 2458576.334 ...  --         -- 107113345
118339710     118339710 6695.01 2458525.646 ...  --         -- 118339710
140215502     140215502       - 2458460.533 ...  --         -- 140215502
140750416     140750416       - 2458504.801 ...  --         -- 140750416
145006304     145006304       - 2458579.505 ...  --         -- 145006304
156716001     156716001       -  2458485.53 ...  --

# Now the main worker function

This function does the following:
-  Load the latest TESS data
-  Flatten the lightcurve
-  Fit the events
-  Calcualte the aliases
-  Plot the permissable aliases
-  Create a report

In [17]:
def func(i):    
    # Create the output dir (we'll use this as cache for the data too)
    output_dir = os.getcwd() + '/{:}'.format(duos['tic_id'][i])
    os.system('mkdir -p {:}'.format(output_dir))
    
    # Do this if it borked and you want to re-run quickly 
    #if os.path.isfile(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i])) : return

    # Now load the TESS data (SPOC, QLP)
    # We are not making our own here like TESSTPF, not yet anyway...
    # for data_type
    #   single_product -> all sectors together
    #   per_sector -> list of per-sector lightcurves
    #   northern_duos -> YEARS 2 and 4, then a list of other sectors 
    #   southern -> YEARS 1 and 3, then a list of other sectors (NOT IMPLEMENTED YET) 
    t, data,data_labels, base_dir =  bruce.ambiguous_period.download_tess_data(duos['tic_id'][i], 
                                                              max_sector=None, 
                                                                   use_ffi=True, 
                                                                   download_dir=None, 
                                                                   bin_length=0.5/24)
    
    # Now flatten the data
    for j, k in zip(data, data_labels):
        # Flatten the data by SG filter, we need an odd kernel legth based on cadence
        j.flatten_data_old(window_width=3, sigmaclip=3, dx_lim=0.1)


#         for seg in bruce.data.find_nights_from_data(j.time, dx_lim=0.2):
#             j.w = np.ones(j.time.shape[0])*np.median(j.flux)

        # Optinally save the data
        j.write_data(output_dir + '/' +'TESS_DATA_{:}.txt'.format(k))
        fig, ax = j.plot_segments(dx_lim=0.5)
        fig.savefig(output_dir + '/' + 'TESS_DATA_{:}.png'.format(k))
        plt.close(fig)
        
        
    # Now we need to adjust the time for TTV cases
    if duos['tic_id'][i]==118339710:
        
        x = np.array([1525.64728,2249.54431,2973.53737,3697.59768])+2457000
        y = np.array([0,-0.087396,-0.078762,-0.002878])
        p_ttv = np.polyfit(x,y, 3)
        def p_ttv(x):
            return 0.08*np.cos(1425.64728 + 2*x*np.pi/(2*(2171.9503999999997)))
        
        for d in data:
            #print(np.polyval(p, d.time))
            #d.time = d.time - np.polyval(p_ttv, d.time)
            d.time = d.time + p_ttv(d.time)
    # # Now re-order_datasets based on epochs given
    # We will unpack now, (data with transits, and data without)
    # We may need to change this for it to work properly (Sam is working on it)
    # Its worth noting we can incorparate ground based data here too
    # data_from_ground = bruce.ambiguous_period.mono_event.photometry_time_series(time, flux, flux_err, w = norm_model)
    # Then this can go in data_other_sectors
    data, data_labels = bruce.ambiguous_period.group_data_by_epochs(data, data_labels, duos['t_zero_1'][i], duos['t_zero_2'][i])
    data, data_other_sectors = data[0], data[1:]



    ############################
    # FIT EVENT 1
    ############################
    # Mask data and create the mono_event object
    nmask = 3
    mask1 = (data.time > (duos['t_zero_1'][i] - nmask*duos['width_1'][i])) &  (data.time < (duos['t_zero_1'][i] + nmask*duos['width_1'][i]))
    #data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], data.flux_err[mask1], w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], median_abs_deviation(data.flux[mask1])*np.ones(mask1.sum(), dtype=np.float64), w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
    m1 = bruce.ambiguous_period.mono_event(duos['t_zero_1'][i], duos['width_1'][i], duos['depth_1'][i], data_event_1, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = None)
    
    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_1 = m1.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)


    ############################
    # FIT EVENT 2
    ############################
    # Mask data and create the mono_event object
    mask2 = (data.time > (duos['t_zero_2'][i] - nmask*duos['width_2'][i])) &  (data.time < (duos['t_zero_2'][i] + nmask*duos['width_2'][i]))
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], data.flux_err[mask2], w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], median_abs_deviation(data.flux[mask2])*np.ones(mask2.sum(), dtype=np.float64), w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
    m2 = bruce.ambiguous_period.mono_event(duos['t_zero_2'][i], duos['width_2'][i], duos['depth_2'][i], data_event_2, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

    # Fit the event and report plots
    fig_initial, ax_initial, fig_final, ax_final, return_data_2 = m2.fit_event_with_fixed_period(fit_period=30., plot=True, )
    fig_initial.tight_layout()
    fig_final.tight_layout()
    fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
    fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
    plt.close(fig_initial); plt.close(fig_final)

    # We are going to make a nice plot of the two events with their models
    fig, ax = plt.subplots(1,2, gridspec_kw={'hspace' : 0, 'wspace' : 0}, figsize = (6.4, 3.8))
    ax[0].errorbar(return_data_1[0], return_data_1[1], yerr=return_data_1[2], fmt='k.', alpha = 0.1)
    ax[0].plot(return_data_1[3], return_data_1[4], c='orange')
    ax[1].errorbar(return_data_2[0], return_data_2[1], yerr=return_data_2[2], fmt='k.', alpha = 0.1)
    ax[1].plot(return_data_2[3], return_data_2[4], c='orange')
    ax[1].set(yticks=[])
    ylim1 = ax[0].get_ylim()
    ylim2 = ax[1].get_ylim()
    ylim = [min(ylim1[0],ylim2[0]), max(ylim1[1], ylim2[1])]
    ax[0].set_ylim(ylim)
    ax[1].set_ylim(ylim)
    fig.supxlabel('Time from Transit [d]', fontsize=18, x=0.55, y = -0.005)
    fig.supylabel('Flux', fontsize=18)
    fig.suptitle(m2.name, y=0.95, x=0.55, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3', alpha=1.0), ha='center', fontsize=18)
    plt.subplots_adjust(right=0.99, top=0.99, bottom=0.13)
    fig.savefig(output_dir + '/' + 'TIC-{:}_BOTH_EVENTS.png'.format(duos['tic_id'][i]))
    plt.close(fig)

    ########################################################
    # CREATE THE AMBIGUOUS PERIOD OBJECT
    ########################################################
    p = bruce.ambiguous_period.ambiguous_period(data, events=[m1,m2], name='TIC-{:}'.format(duos['tic_id'][i]),
                        median_bin_size = 2,convolve_bin_size = 2)

    # Now mask and filter 
    p.mask_and_filter_events()

    # Calculate aliases
    # Do not use nsolutions_events here (that is superceeded later)
    nsolutions_events = p.calcualte_aliases(dx_lim=0.03, min_period=15)

    # Now calcualte whether we saw a transit by comparing the model to a flat line
    delta_L_data = p.calcualte_data_delta_L(data)
    
    
    ########################################################
    # CHECK NGTS PHOTOMETRY
    ########################################################
    ngts_data = []
    ngts_data_labels = []
    if os.path.isfile('ngts_data/TIC-{:}.fits'.format(duos['tic_id'][i])):
        ngts = Table.read('ngts_data/TIC-{:}.fits'.format(duos['tic_id'][i]), hdu=4)
        ngts.sort('BJD')
        mask = (ngts['SIGMA_XS']<0.02) & ~np.isnan(np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64)) &\
                     ~np.isnan(np.array(ngts['FLUX_ERR'], dtype=np.float64)) &\
                     ~np.isinf(np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64)) &\
                     ~np.isinf(np.array(ngts['FLUX_ERR'], dtype=np.float64))
        ngts = ngts[mask]
        
        # Now lets bin
        t_bin, f_bin, fe_bin, c = bruce.data.bin_data(np.array(ngts['BJD'], dtype=np.float64),
                                                  np.array(ngts['FLUX_SYSREM_ALIGNED'], dtype=np.float64),
                                                  0.5/24/3)

        t_bin, f_bin, fe_bin = t_bin[c>10], f_bin[c>10], fe_bin[c>10]
        if len(t_bin)>3 : 
            
            #######################################################################################
            #    OK, there are two ways of doing this based on data volume
            # We could 
            # 1. Split each night into its own LC so we can see how ruling out aliases
            #    progresses. This can be cumbersome and make large plots.
            # 2. Treat the NGTS LC as one, but if wee see a transit we will have 
            #    to do a retrospective calculation to see what aliases is was compatible with.
            #    It also might bork the delta_L calculation if a transit is seen but later excluded.
            #    Furthermore, if we only have a short amount of in-transit data, we have to be
            #    careful about how we detrend.
            #######################################################################################

            
#             #######################################################################################
#             # Aproach 1
#             #######################################################################################

#             # Now create the data products 
#             for seg in bruce.data.find_nights_from_data(t_bin, dx_lim=0.2): 
#                 # Now check the phase is consistent with one of the aliases

#                 # Now get the phases of all aliases
#                 phases = np.zeros((p.aliases.shape[0], len(np.array(t_bin, dtype=np.float64))))
#                 for j in range(len(phases)):
#                     phases[j] = bruce.data.phase_times(t_bin, p.events[0].de_get_epoch(), p.max_period/p.aliases[j], phase_offset=0.2)
#                 phase_widths = p.events[0].de_transit_width() / (p.max_period/p.aliases)

#                 # Now mask the data which doesent fall within 1 width of any alias
#                 useful_data = np.abs(phases) < (phase_widths[:,np.newaxis]/2) # Make for a nice plot

#                 # If so, lets make a data object for the night 
#                 if useful_data.any():
#                     # Now create the time series object
#                     d = bruce.ambiguous_period.photometry_time_series(t_bin[seg], f_bin[seg], fe_bin[seg])
#                     d.w = np.ones(len(d.time))

#                     # Now normalise to the median of the top 20%
#                     try:
#                         quarter_edges = np.linspace(np.min(d.time)-1e-3,np.max(d.time)+1e-3,5)
#                         dig = np.digitize(d.time, quarter_edges, right=True)
#                         medians = np.array([np.nanmedian(d.flux[dig==j]) for j in range(1,len(quarter_edges))])
#                         constant = np.nanmax(medians)
#                     except :  constant = np.nanmedian(d.flux)


#                     d.flux = d.flux / constant
#                     d.flux_err = d.flux_err / constant
#                     ngts_data.append(d)

#                     ngts_data_labels.append(Time(d.time[0], format='jd').datetime.strftime('%Y-%m-%d'))            
#                     print(Time(d.time[0], format='jd').datetime.strftime('%Y-%m-%d'))


            #######################################################################################
            # Aproach 2
            #######################################################################################
            # Now create the time series object
            d = bruce.ambiguous_period.photometry_time_series(t_bin, f_bin, fe_bin)
            d.w = np.ones(len(d.time))
            
            # We will do a crude variability removal
            # BE CAREFUL, only a small amount of in-transit data will 
            for seg in bruce.data.find_nights_from_data(d.time, dx_lim=0.2): 
                d.w[seg] = np.nanmedian(d.flux[seg])*np.ones(seg.shape[0])
            ngts_data.append(d)
            ngts_data_labels.append('NGTS data')
            
        # Now we need to adjust the time for TTV cases
        if duos['tic_id'][i]==118339710:
            for d in ngts_data:
                #print(len(d.time), d.time)
                #print(np.polyval(p, d.time))
                #d.time = d.time - np.polyval(p_ttv, d.time)
                d.time = d.time + p_ttv(d.time)
    
    ########################################################
    # CALCULATE DELTA L
    ########################################################
    #delta_L_data = delta_L_data + 100 # THIS FUDGE IS OFTEN NEEDED
#     for j in range(delta_L_data.shape[0]):
#         print(j, delta_L_data[j])
    delta_L_data_from_other_sectors_or_others = [p.calcualte_data_delta_L(j) for j in data_other_sectors]
    delta_L_data_from_ngts = [p.calcualte_data_delta_L(j) for j in ngts_data]
    p.delta_L = np.array([delta_L_data, *delta_L_data_from_other_sectors_or_others, *delta_L_data_from_ngts])

    ########################################################
    # Plot the aliases
    ########################################################
    fig, ax  = p.plot_aliases(phot_data=[*data_other_sectors, *ngts_data], 
                              phot_data_labels=[*data_labels, *ngts_data_labels])
    fig.savefig(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i]), dpi=300)
    plt.close(fig)
    
    ########################################################
    # Now report the aliases
    ########################################################
    aliases = (p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    periods = (p.max_period/p.aliases)[p.alias_mask[:,-1]==p.alias_mask.max()]
    alaises = Table()
    alaises.add_column(Column(duos['tic_id'][i]*np.ones(len(aliases), dtype=int), name='tic_id'))
    alaises.add_column(Column(m1.de_get_epoch()*np.ones(len(aliases)), name='t_zero_1'))
    alaises.add_column(Column(m1.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_1'))
    alaises.add_column(Column(m1.de_get_k()*np.ones(len(aliases)), name='k_1'))
    alaises.add_column(Column(m1.de_get_b()*np.ones(len(aliases)), name='b_1'))
    alaises.add_column(Column(m1.de_transit_width()*np.ones(len(aliases)), name='width_1'))
    alaises.add_column(Column(m2.de_get_epoch()*np.ones(len(aliases)), name='t_zero_2'))
    alaises.add_column(Column(m2.de_get_radius_1()*np.ones(len(aliases)), name='radius_1_2'))
    alaises.add_column(Column(m2.de_get_k()*np.ones(len(aliases)), name='k_2'))
    alaises.add_column(Column(m2.de_get_b()*np.ones(len(aliases)), name='b_2'))
    alaises.add_column(Column(m2.de_transit_width()*np.ones(len(aliases)), name='width_2'))
    alaises.add_column(Column(aliases, name='alias'))
    alaises.add_column(Column(periods, name='period'))
    alaises.write(output_dir + '/' + 'TIC-{:}_ALIASES.fits'.format(duos['tic_id'][i]), overwrite=True)

    
    ########################################################
    # Now plan the aliases
    ########################################################
#     transit_events = p.transit_plan(start=Time.now(), end = Time.now()+TimeDelta(30, format='jd'), resolution = 1*u.minute,
#                     tic_id=duos['tic_id'][i], observatory='Paranal',
#                     sun_max_alt=-15, target_min_alt=30, moon_min_seperation=20,
#                     min_time_in_transit=None, min_frac_in_transit=None)
#     transit_events.add_column(Column(np.ones(len(transit_events))*m1.depth, name='transit_depth_1'))
#     transit_events.add_column(Column(np.ones(len(transit_events))*m2.depth, name='transit_depth_2'))
#     transit_events.add_column(Column(np.ones(len(transit_events))*m1.width, name='transit_width_1'))
#     transit_events.add_column(Column(np.ones(len(transit_events))*m2.width, name='transit_width_2'))
#     transit_events.add_column(Column(np.ones(len(transit_events),dtype=int)*len(alaises), name='naliasesleft'))
#     transit_events.write(output_dir + '/' + 'TIC-{:}_ALIASES_WINDOWS_PARANAL.fits'.format(duos['tic_id'][i]), overwrite=True)
#     p.plot_all_events(transit_events, output_dir=output_dir)

    line = str(m1.de_get_epoch()) + ','
    line += str(30) + ','
    line += str(m1.de_get_radius_1()) + ','
    line += str(m1.de_get_k()) + ','
    line += str(m1.de_get_b()) + ','
    line += str(m1.de_transit_width()*24) + ','   
    line += str(m1.depth) + ',,'   # R2 will do later 
    
    line += str(m2.de_get_epoch()) + ','
    line += str(30) + ','
    line += str(m2.de_get_radius_1()) + ','
    line += str(m2.de_get_k()) + ','
    line += str(m2.de_get_b()) + ','
    line += str(m2.de_transit_width()*24) + ','   
    line += str(m2.depth) + ',,'   # R2 will do later 
    
    line += '"' + ','.join([str(i) for i in alaises['alias']]) + '"'
    
    with open(output_dir + '/' + 'TIC-{:}_gs.txt'.format(duos['tic_id'][i]), 'w+') as f:
        f.write(line)

    plt.close()
    gc.collect()


import multiprocess
with multiprocess.Pool(6) as p : p.map(func, range(len(duos)))
    


            
            
# f = open('failed.txt', 'w+')
# f.close()
# for i in range(len(duos))[5:]: 
#     try : func(i)
#     except: 
#         with open('failed.txt', 'a+') as f:
#             f.write('{:}\n'.format(duos['tic_id'][i]))

            
# print(np.argwhere(duos['tic_id']==126565211)[0][0])   
#func( np.argwhere(duos['tic_id']==140215502)[0][0] )




📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpdndbf2qo


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpu19nylo_


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp9e7emqlw


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpstkma3re


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp4vonl7jy


📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmpf880xmr2

Querying TIC 159490807 from MAST...
Querying TIC 20904104 from MAST...
Querying TIC 231637303 from MAST...
Querying TIC 140750416 from MAST...
Querying TIC 107113345 from MAST...
Querying TIC 67599025 from MAST...


 [Done]


 [Done]

WARNING

: UnitsWarning: 'pixels' did not parse as fits unit: At col 0, Unit 'pixels' not supported by the FITS standard. Did you mean pixel? If this is meant to be a custom unit, define it with 'u.def_unit'. To have it recognized inside a file reader or other code, enable it with 'u.add_enabled_units'. For details, see https://docs.astropy.org/en/latest/units/combining_and_defining.html [astropy.units.core]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:42<00:00,  1.14it/s]



📂 Download directory: /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp82rkhudz

Querying TIC 302728777 from MAST...


/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/data_retrieval.py:292: RuntimeWarning: All-NaN slice encountered
  if np.nanmedian(corrected_flux)>100 :
/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/data_retrieval.py:319: RuntimeWarning: All-NaN slice encountered
  print('MEDIAN CHECK not met : ' + str(np.nanmedian(corrected_flux)))


MEDIAN CHECK not met : nan

✅ Download summary:
Sector SPOC FFI QLP LC TESS-SPOC LC SPOC LC     Source                                                                File                                                            
------ -------- ------ ------------ ------- ------------- ----------------------------------------------------------------------------------------------------------------------------
    21       --     --            X      --  TESS-SPOC LC /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp82rkhudz/hlsp_tess-spoc_tess_phot_0000000302728777-s0021_tess_v1_lc.fits
    48       --     --            X      --  TESS-SPOC LC /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp82rkhudz/hlsp_tess-spoc_tess_phot_0000000302728777-s0048_tess_v1_lc.fits
    75        X     --           --      -- FFI custom LC         /var/folders/fs/c4gpqhmx5tbcbsv_cbs0yhn80000gn/T/tmp82rkhudz/tess-s0075-1-4_169.508869_54.282411_10x10_astrocut.fits

Temporary data stored in: /var/folde

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  8.29it/s]
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '
/Users/sam/anaconda3/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [ ]:
data

# Now look for solved systems

In [ ]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p solved_systems; rm solved_systems/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==1:
        os.system('cp {:}/TIC-{:}_ALIASES.png solved_systems'.format(tic_id,tic_id))
        
        print('{:}, {:}, {:}'.format(tic_id,  t['t_zero_1'][0], t['period'][0]))
        

# Now look for ones with no aliases left (to debug)

In [ ]:
import glob
files = glob.glob('*/TIC-*_ALIASES.fits')

os.system('mkdir -p no_aliases; rm no_aliases/*')

for i in range(len(files)):
    tic_id  = int(files[i].split('/')[0])
    t = Table.read(files[i])
    if len(t)==0:
        os.system('cp {:}/TIC-{:}_ALIASES.png no_aliases'.format(tic_id,tic_id))
        print(tic_id)

# Now look for ones that did not predict

In [ ]:
for i in range(len(duos)):
    if not os.path.isfile('{:}/TIC-{:}_ALIASES_WINDOWS_PARANAL.fits'.format(duos['tic_id'][i], duos['tic_id'][i])):
        print(duos['tic_id'][i])

# Now make predictions about when they will transit

In [4]:
from astropy.table import Table, vstack
import glob

# Read all files and combine them
files = glob.glob('*/TIC-*_ALIASES_WINDOWS_PARANAL.fits')
events = vstack([Table.read(f) for f in files], join_type='outer')

# Group by tic_id and night
grouped = events.group_by(['tic_id', 'night'])

rows = []
for g in grouped.groups:
    # Start with the first row's values for all columns
    row = {col: g[col][0] for col in events.colnames}

    # Combine aliasP and aliasPer values as comma-separated strings
    row['aliasP'] = ','.join(str(v) for v in g['aliasP'])
    row['aliasPer'] = ','.join('{:.5f}'.format(v) for v in g['aliasPer'])
    
    rows.append(row)

# Create summary table
summary = Table(rows)
summary.sort('night')
print(summary['night', 'tic_id', 'aliasP', 'aliasPer'])

  night      tic_id      aliasP                      aliasPer                  
---------- --------- ------------- --------------------------------------------
2025-11-05 140750416         19,39                            37.02956,18.51478
2025-11-05 332657786            39                                     36.55523
2025-11-05 304339227            16                                     42.04042
2025-11-05  42428568            16                                     44.03737
2025-11-05 251057075            28                                     25.09342
2025-11-06  20904104            45                                     15.54780
2025-11-06  71272316      14,29,44                   73.68288,36.84144,24.56096
       ...       ...           ...                                          ...
2025-12-03 251057075 8,17,26,35,44 80.85658,40.42829,26.95219,20.21415,16.17132
2025-12-03 156716001            18                                     38.89929
2025-12-03 107113345            37      

# Now plot for a night

In [5]:
night = '2025-11-07'

t = summary[summary['night']==night]
print(t['night', 'tic_id', 'aliasP', 'aliasPer', 'transit_depth_1', 'transit_width_1'])
p = bruce.ambiguous_period.ambiguous_period('a', events=['a','a'], name='dummy',median_bin_size = 2,convolve_bin_size = 2)
p.plot_all_events(t, output_dir='transit_plan')

  night      tic_id  aliasP      aliasPer      transit_depth_1 transit_width_1
---------- --------- ------ ------------------ --------------- ---------------
2025-11-07 332697924     16           43.08654    0.0015688539      0.20261718
2025-11-07 434452008  16,50 128.82851,42.94284  0.002670359474     0.413639352
2025-11-07  20904104  18,37  37.64205,18.82103    0.0009678006      0.30561772
2025-11-07 251057075     24           29.10837    0.0023933053      0.15875384


/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/ambiguous_period.py:633: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.plot_date([mid_time_plot], [mid_alt],
/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/ambiguous_period.py:633: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.plot_date([mid_time_plot], [mid_alt],
/Users/sam/anaconda3/lib/python3.9/site-packages/bruce-1.0.0-py3.9-macosx-10.9-x86_64.egg/bruce/ambiguous_period/ambiguous_period.py:633: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "o" (-> marker='o'). The keyword argument will take precedence.
  ax.plot_date([mid_tim

# monos sheet

In [4]:
duos

TIC ID,Linked TIC ID,TOI ID,t_zero_1,width_1,depth_1,t_zero_2,width_2,depth_2,RA (decimal deg),Dec (decimal deg),Sector of first transit,Sector of second transit,T0 (BTJD),T0_uncertainty (days),Period (days),Period_uncertainty (days),Duration (hrs),Duration_uncertainty (hrs),Depth (ppm),Depth_uncertainty (ppm),R_planet (R_Earth),Active_Aliases,NGTS data? /comment,Recent TESS sectors -check LC,Upcoming TESS sectors,RV - Solène notes,Simbad,Papers,SG1,Published?,tic_id
int64,int64,str7,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,str209,str3,str29,str23,str214,str23,str320,int64,int64,int64
20904104,20904104,-,2458390.178,0.30561772,0.0009678006,2459105.339,0.15087725,0.0013742447,7.413684822,-19.36625377,3,29,1390.178,0.00078,715.161,0.00131,0.30562,0.05199,968,7.80773,2.41271,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43",yes,-,-,3 CORALIE vetting passed. if period is solved we could continue the RV follow-up,Star,"Kordopatis+2013 [2013AJ....146..134K],Kunder+2017 [2017AJ....153...75K],Ruiz-Dern+2018 [2018A&A...609A.116R],Ratzenböck+2020 [2020A&A...639A..64R],Steinmetz+2020 [2020AJ....160...83S]",--,--,20904104
22317640,22317640,-,2458589.378,0.1718912471,0.01121437078,2460064.808,0.1946916101,0.01201410494,183.9093228,-47.19813113,10,64,1589.378,0.00041,1475.43,0.00054,0.17189,0.05483,11214,86.21509,12.03593,"1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47",yes,-,"99,100,101,112,113","WINE (FEROS 5, HARPS 13 from 2022)",Star,Malik+2025 [2025AJ....170...39M],--,--,22317640
42428568,42428568,-,2458475.69,0.39174843,0.0052972436,2459224.322,0.3060124,0.0048009553,96.29346696,-3.766571869,6,33,1475.69,0.00056,748.632,0.00116,0.39175,0.05482,5297,41.07723,18.46309,"1,2,3,4,5,6,7,8,9,10,11,12,13,17,23,24,25,26",yes,-,"110,111","CORALIE (2): RV difference is 360 m/s in one week the RV error is about 80 m/s, likely binary star",Star,Buder+2021 [2021MNRAS.506..150B],--,--,42428568
67599025,67599025,-,2458404.028,0.12480073,0.0076516867,2459138.825,0.1358673,0.0076302686,17.69268173,-31.50549356,3,30,1404.028,0.00026,734.797,0.00034,0.1248,0.01978,7652,25.64378,6.86574,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31",yes,-,"104,106,107",WINE (FEROS 35 from 2020 to 2023),--,--,--,--,67599025
71028120,71028120,-,2458445.866,0.14199898,0.001283884,2459189.955,0.20965792,0.0010778109,66.12741053,-11.98354302,5,32,1445.866,0.00031,744.089,0.00077,0.142,0.01092,1284,4.89114,4.59889,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28",yes,-,"98,109",--,High Proper Motion Star,"Siebert+2011 [2011AJ....141..187S],Boeche+2011 [2011AJ....142..193B],Coşkunoǧlu+2012 [2012MNRAS.419.2844C],Kordopatis+2013 [2013AJ....146..134K],Francis+2013 [2013MNRAS.436.1343F],Munari+2014 [2014AJ....148...81M],Kunder+2017 [2017AJ....153...75K],Steinmetz+2020 [2020AJ....160...83S],Järvinen+2025 [2025A&A...698A..93J]",--,--,71028120
71272316,71272316,-,2459070.725,0.08676289,0.0063369274,2460175.967,0.105793186,0.0063734055,330.1327946,-44.71586765,28,68,2070.725,0.00034,1105.242,0.00069,0.08676,0.04101,6337,70.02135,8.5827,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,78",yes,-,"105,106",--,Star,--,--,--,71272316
107113345,107113345,-,2458576.334,0.1923165369,0.00227545153,2459317.299,0.09514047402,0.002114618932,183.0341034,-32.66283108,10,37,1576.334,0.00042,740.965,0.00047,0.19232,0.02782,2275,15.66661,6.68057,--,yes,90 - another transit?,"101,114",--,Star,"Kordopatis+2013 [2013AJ....146..134K],Kunder+2017 [2017AJ....153...75K],Steinmetz+2020 [2020AJ....160...83S]",--,--,107113345
118339710,118339710,6695.01,2458525.646,0.3978936449,0.004337083823,2459249.547,0.3824385339,0.003083043331,129.5720399,-23.55022929,8

In [21]:
lines = []
for i in duos['tic_id']:
    with open('{:}/TIC-{:}_gs.txt'.format(i,i), 'r') as f : s = f.readlines()[0]
    print(str(i) + ',' + s)

    


20904104,2458390.1566440808,30,0.04090226119891004,0.026910075463958003,0.5387560526187379,8.19890903849551,0.0009678006,,2459105.3569135834,30,0.09744769820899687,0.289890907933557,1.2539087934190045,6.808782523984608,0.0013742447,,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47"
22317640,2458589.3781096577,30,0.016388534511783348,0.1024542822463799,0.19714650386647836,4.074281152140242,0.01121437078,,2460064.8085718504,30,0.026733789768421737,0.11232069829289366,0.7677522221248709,4.932797644460483,0.01201410494,,"1,3,5,7,9,11,13,15,17,19,21,23,25,29,33,35,37,39,41"
42428568,2458475.689948855,30,0.07850260609764798,0.08331666118178495,0.9468186676120407,9.49991416950761,0.0052972436,,2459224.3257045094,30,0.09600593805013705,0.29144257866208423,1.2169442332995115,9.579928664144143,0.0048009553,,"1,2,3,4,5,6,7,8,9,10,11,12,13,17,23,24,25,26"
67599025,2458404.022274778,30,0.0140693727194173,0.080012302

In [13]:
import re

s = '2459105.3549437337,30,0.02613716879930225,0.03436973297992696,0.6599782293154693,4.77201133826509,0.0013742447,,"1",2",3",4",5",6",7",8",9",10",11",12",13",14",15",16",17",18",19",20",21",22",23",24",25",26",27",28",29",30",31",32",33",34",35",36",37",38",39",40",41",42",43",44",45",46",47"'

# Step 1: find the first and last quote positions
first = s.find('"')
last = s.rfind('"')

# Step 2: remove all other quotes in between
if first != -1 and last != -1 and last > first:
    inner = s[first+1:last].replace('"', '')
    s = s[:first+1] + inner + s[last:]

print(s)


2459105.3549437337,30,0.02613716879930225,0.03436973297992696,0.6599782293154693,4.77201133826509,0.0013742447,,"1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47"
